# Debugging PiNN layers and networks [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Teoroo-CMC/PiNN/blob/master/docs/notebooks/Layer_debug.ipynb)

In [1]:
!pip install git+https://github.com/Teoroo-CMC/PiNN
!wget -nv -nc https://raw.githubusercontent.com/Teoroo-CMC/PiNN_lab/master/resources/qm9_train.{yml,tfr}

  Cloning https://github.com/Teoroo-CMC/PiNN to /local/tmp.2181992/pip-req-build-gv24p52j
  Running command git clone -q https://github.com/Teoroo-CMC/PiNN /local/tmp.2181992/pip-req-build-gv24p52j
     |████████████████████████████████| 738 kB 6.2 MB/s eta 0:00:01
  Created wheel for pinn: filename=pinn-1.2.0.dev0-py3-none-any.whl size=54139 sha256=e32c7028fff04705998ac3d7a800f872b7b5617fa742282aec12239688b21e42
  Stored in directory: /local/tmp.2181992/pip-ephem-wheel-cache-kxuviaek/wheels/8d/bf/2a/c323e21da6d959e1377d7e0f1206fde631ebc1f5947defc463
Successfully built pinn
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'METADATA'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the '/apps/Arch/software/Python/3.9.5-GCCcore-10.3.0/bin/python -m pip install --upgrade pip' command.
2024-04-08 11:07:3

## Loading data

For the purpose of testing we download a subset of the QM9 dataset used in [PiNN_lab](https://github.com/Teoroo-CMC/PiNN_lab).

In [2]:
from pinn.io import load_tfrecord, sparse_batch

dataset = load_tfrecord("qm9_train.yml").apply(sparse_batch(10))
for datum in dataset:
    print({k: v.shape for k, v in datum.items()})
    break

{'A': TensorShape([10]), 'B': TensorShape([10]), 'C': TensorShape([10]), 'Cv': TensorShape([10]), 'G': TensorShape([10]), 'H': TensorShape([10]), 'U': TensorShape([10]), 'U0': TensorShape([10]), 'alpha': TensorShape([10]), 'coord': TensorShape([176, 3]), 'elems': TensorShape([176]), 'gap': TensorShape([10]), 'homo': TensorShape([10]), 'lumo': TensorShape([10]), 'mu': TensorShape([10]), 'r2': TensorShape([10]), 'zpve': TensorShape([10]), 'ind_1': TensorShape([176, 1])}


## Using PiNN Layers

PiNN networks and layers are Keras [Layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer) 
and [Models](https://www.tensorflow.org/api_docs/python/tf/keras/Model). 

To use them, you create an instance of layer, after that, the layer object can be used as a function. Each layer
is initialized with different parameters and requires different input tensors, see their individual documentation
for the details.

In [3]:
from pinn.layers import CellListNL
from pinn.networks.pinet import PILayer, PiNet

nl = CellListNL(rc=5)

for datum in dataset:
    nl(datum)
    break

The definition of a layer needs three parts:

- `__init__` defines the layer object; 
- `build` creates the necessary variables or sub-layers; 
- `call` defines how the input tensors are processed.

The `build()` method is only called once when the layer is used for the first tiem (e.g. in a loop).
See below for an example definition for the `PILayer`

In [4]:
??PILayer

Init signature: PILayer(*args, **kwargs)
Source:        
class PILayer(tf.keras.layers.Layer):
    """PiNN style interaction layer

    Args:
        n_nodes: number of nodes to use
            Note that the last element of n_nodes specifies the dimention of
            the fully connected network before applying the basis function.
            Dimension of the last node is [pairs*n_nodes[-1]*n_basis], the
            output is then summed with the basis to form the interaction nodes
        **kwargs: keyword arguments will be parsed to the feed forward layers
    """
    def __init__(self, n_nodes=[64], **kwargs):
        super(PILayer, self).__init__()
        self.n_nodes = n_nodes
        self.kwargs = kwargs

    def build(self, shapes):
        self.n_basis = shapes[2][-1]
        n_nodes_iter = self.n_nodes.copy()
        n_nodes_iter[-1] *= self.n_basis
        self.ff_layer = FFLayer(n_nodes_iter, **self.kwargs)

    def call(self, tensors):
        ind_2, prop, basis = tensor

## Using PiNN Networks

`network` (Keras Models) are defined similarly, but they can be directly used to perform regression task.

By default, `network`  produces per-atom predictions, this can be changed by the `out_pool` parameter to 
get some simple per-structure predictions. In that case, the network object can be used to perform trainig directly.



In [6]:
def label_data(data):
    # defines the label to train on
    x = data
    y = data['lumo']
    return x, y

train = dataset.map(label_data)
pinet = PiNet(out_pool='min')
pinet.compile(optimizer='Adam', loss='MAE')
pinet.fit(train, epochs=3)

Epoch 1/3


/home/yunqi/.miniconda/envs/pinn-tf2/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/concat_1:0", shape=(None,), dtype=int32), values=Tensor("Adam/gradients/concat:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/pi_net_1/gc_block_7/pi_layer_7/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/home/yunqi/.miniconda/envs/pinn-tf2/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/concat_3:0", shape=(None,), dtype=int32), values=Tensor("Adam/gradients/concat_2:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/pi_net_1/gc_block_6/pi_layer_6/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a

2000/2000 [==============================] - 37s 17ms/step - loss: 0.0551
Epoch 2/3
2000/2000 [==============================] - 35s 18ms/step - loss: 0.0285
Epoch 3/3
2000/2000 [==============================] - 35s 17ms/step - loss: 0.0240


## Further benchmarks

For more advanced usage you are recommended to use the `Model` API to define the trainig loss, derived predicates.
For traininig potential energy surfaces, you are recommended to use `pinn.models.potential_model` in combination with the command line interface (CLI).

Alternatively, see the [Trainig Tips](More_on_training.ipynb) notebook to see how to run the tranining interactively in a notebook.